In [5]:
from bs4 import BeautifulSoup as bs
import re 
import os
import lxml
import xml.etree.ElementTree as ET
from lxml import objectify
from datetime import datetime
import numpy as np 
import pandas as pd 
import requests
import ftplib
from io import BytesIO
import tqdm

### Prepare to load data

In [3]:
'''
with open(file) as f:
    parsed = objectify.parse(f)
root = parsed.getroot()
'''

'\nwith open(file) as f:\n    parsed = objectify.parse(f)\nroot = parsed.getroot()\n'

##### Read xml file 

In [2]:
file = "20091_M100_helemoedet.xml"
tree = ET.parse(file)
root = tree.getroot()

FileNotFoundError: [Errno 2] No such file or directory: '20091_M100_helemoedet.xml'

##### Subelements / children

In [ ]:
# children of the root only 
root_children = []

for child in root:
    root_children.append((child.tag, child.attrib))

In [ ]:
# All children in file
all_children = [elem.tag for elem in root.iter()]

In [29]:
unique_children = set(all_children)
print(unique_children)

NameError: name 'all_children' is not defined

# Single file (lokal)

In [14]:
# entire xml file as string
xml = indhold[0].tostring(root, encoding='utf8').decode('utf8')

AttributeError: 'bytes' object has no attribute 'tostring'

# Online (Kim)

In [6]:
ftp_url = "oda.ft.dk"
ftp_dir = "/ODAXML/Referat/samling/"

# Initialisering af lister
### Jeg skal finde ud af, hvordan jeg dropper dem igen, når jeg er færdig

In [7]:
dirs = []
sub_dir = []
filer = []
filer_navne = []
sti_navn = []
indhold = []

suppe = []

### Login til ftp serveren og henter en liste over biblioteker

In [8]:
ftp = ftplib.FTP(ftp_url)
ftp.login("anonymous", "wpg345@alumni.ku.dk")
ftp.set_pasv(True)
ftp.cwd(ftp_dir)
ftp.dir(dirs.append)

### Bygger en liste med underbibliotekter

In [9]:
for i in range (0, len(dirs), 1):
    sub_dir.append(ftp_dir+dirs[i][-5:]+'/') 

### Bygger en liste med filnavne

In [10]:
for i in range (0, len(sub_dir), 1):
    ftp.cwd(sub_dir[i])
    ftp.dir(filer.append)

### Ekstrakt af filnavn

In [11]:
for i in range(0, len(filer)):
    filer_navne.append(filer[i][39:])

### Opbygning af hele stien og filnavn

In [12]:
for i in range(0, len(filer_navne)):
    sti_navn.append(ftp_dir+filer_navne[i][0:5]+'/'+filer_navne[i])

### Overførsel af de enkelte filer fra ftp://oda.ft.dk, og indholdet skrives til liste

In [13]:
for i in tqdm.tqdm(range(0, len(sti_navn))):
# for i in tqdm.tqdm(range(0, 10)):
    r = BytesIO()
    ftp.retrbinary(f"RETR {sti_navn[i]}", r.write)
    indhold.append(r.getvalue())
    r.close()

100%|██████████| 1489/1489 [02:51<00:00,  8.67it/s]


# Single file (online)

##### Soup 

In [16]:
xml = indhold[0]

In [17]:
soup = bs(xml, 'lxml')

In [18]:
soup.find_all("dagsordenpunkt")

[<dagsordenpunkt><metaftagendaitem><itemno>0</itemno><ftcase></ftcase><ftcasenumber></ftcasenumber><ftcasetype>FM</ftcasetype><ftcasestage></ftcasestage><shorttitle>Punkt 0</shorttitle></metaftagendaitem><punkttekst><exitus><linea><char></char></linea></exitus></punkttekst><aktivitet><rubrica><linea><char>Meddelelser fra formanden</char></linea></rubrica><tale><taler><metaspeakermp><oratorfirstname>Holger K.</oratorfirstname><oratorlastname>Nielsen</oratorlastname><groupnameshort>SF</groupnameshort><oratorrole>formand</oratorrole></metaspeakermp><talertitel><linea><char>Tredje næstformand</char><char> (Holger K. Nielsen)</char></linea></talertitel></taler><talesegment><metaspeechsegment><lastmodified>2011-01-18T14:15:16</lastmodified><edixistatus>Typeset</edixistatus><startdatetime>2010-05-31T10:00:02</startdatetime><enddatetime>2010-05-31T10:00:26</enddatetime></metaspeechsegment><tekstgruppe><exitus><linea><char>Mødet er åbnet.</char></linea></exitus></tekstgruppe><tekstgruppe><exitu

In [19]:

'''    
    for dp in range(len(dp_list)):
            temp = []
            temp.append(dp_list[dp].get_text(separator = ' '))
            
            content.append(''.join(temp))
'''

"    \n    for dp in range(len(dp_list)):\n            temp = []\n            temp.append(dp_list[dp].get_text(separator = ' '))\n            \n            content.append(''.join(temp))\n"

# content fixed single

In [3]:
sample = indhold[0]

NameError: name 'indhold' is not defined

In [30]:
sentences_text = []
content = []

In [1]:
def extract_content(xml): 
    soup = bs(xml, 'lxml')
    dagsordenpunkter = soup.find_all("dagsordenpunkt") # liste med dagsordenpunkter
    
    for dp in dagsordenpunkter: # for hvert dagsordenpunkt, dp, findes raw xml sætninger
        sentences_raw = dp.find_all("linea") # gem raw sætninger til liste
        
        for sr in sentences_raw: # for hver raw sætning, kør .text af sætningen
            sentences_text.append(sr.get_text(separator = ' ')) # gem clean sætninger til liste
            content.append(' '.join(sentences_text)) # join, strip og append clean sætninger til liste


In [2]:
extract_content(sample)

NameError: name 'sample' is not defined

In [20]:
dagspunkt_list = soup.find_all("dagsordenpunkt") # liste med dagsordenpunkter

In [21]:
dagspunkt_list[0]

<dagsordenpunkt><metaftagendaitem><itemno>0</itemno><ftcase></ftcase><ftcasenumber></ftcasenumber><ftcasetype>FM</ftcasetype><ftcasestage></ftcasestage><shorttitle>Punkt 0</shorttitle></metaftagendaitem><punkttekst><exitus><linea><char></char></linea></exitus></punkttekst><aktivitet><rubrica><linea><char>Meddelelser fra formanden</char></linea></rubrica><tale><taler><metaspeakermp><oratorfirstname>Holger K.</oratorfirstname><oratorlastname>Nielsen</oratorlastname><groupnameshort>SF</groupnameshort><oratorrole>formand</oratorrole></metaspeakermp><talertitel><linea><char>Tredje næstformand</char><char> (Holger K. Nielsen)</char></linea></talertitel></taler><talesegment><metaspeechsegment><lastmodified>2011-01-18T14:15:16</lastmodified><edixistatus>Typeset</edixistatus><startdatetime>2010-05-31T10:00:02</startdatetime><enddatetime>2010-05-31T10:00:26</enddatetime></metaspeechsegment><tekstgruppe><exitus><linea><char>Mødet er åbnet.</char></linea></exitus></tekstgruppe><tekstgruppe><exitus

In [4]:
sentences_xml = dagspunkt_list[1].find_all("linea")

NameError: name 'dagspunkt_list' is not defined

In [23]:
sentences_xml

[<linea><char></char></linea>,
 <linea><char>Meddelelser fra formanden</char></linea>,
 <linea><char>Tredje næstformand</char><char> (Holger K. Nielsen)</char></linea>,
 <linea><char>Mødet er åbnet.</char></linea>,
 <linea><char>Finansudvalget har afgivet:</char></linea>,
 <linea><char>Betænkning og indstilling vedrørende forslag til folketingsbeslutning i henhold til grundlovens § 47 med hensyn til statsregnskabet for finansåret 2008.</char></linea>,
 <linea><char>(Beslutningsforslag nr.</char></linea>,
 <linea><char>B 248).</char></linea>,
 <linea><char>Betænkningen og indstillingen vil fremgå af www.folketingstidende.dk.</char></linea>]

In [90]:
sentences = []

In [87]:
for sentence in sentences_xml:
    sentences.append(sentence.get_text(separator = ' '))
    con = ' '.join(sentences)

In [89]:
con

' Meddelelser fra formanden Tredje næstformand  (Holger K. Nielsen) Mødet er åbnet. Finansudvalget har afgivet: Betænkning og indstilling vedrørende forslag til folketingsbeslutning i henhold til grundlovens § 47 med hensyn til statsregnskabet for finansåret 2008. (Beslutningsforslag nr. B 248). Betænkningen og indstillingen vil fremgå af www.folketingstidende.dk.'

# content fixed single//

In [58]:
dagspunkt_list = soup.find_all("dagsordenpunkt") # liste med dagsordenpunkter

for dagspunkt in dagspunkt_list:
    sentence_list = dagspunkt.find_all("linea") # liste med sætningerne

    for sentence in range(len(sentence_list)): # for hver sætning, get_text()
        temp = []
        temp.append(sentence_list[sentence].get_text(separator = ' '))
        content.append(''.join(temp))

### Remove datetime tags (lokal)

In [18]:
'''
for tag in soup.find_all('startdatetime'):
    tag.replace('')
    
for tag in soup.find_all('enddatetime'):
    tag.replace('')

for tag in soup.find_all('lastmodified'):
    tag.replace('')
    
for tag in soup.find_all('edixistatus'):
    tag.replace('')
'''

"\nfor tag in soup.find_all('startdatetime'):\n    tag.replace('')\n    \nfor tag in soup.find_all('enddatetime'):\n    tag.replace('')\n\nfor tag in soup.find_all('lastmodified'):\n    tag.replace('')\n    \nfor tag in soup.find_all('edixistatus'):\n    tag.replace('')\n"

In [19]:
'''
soup.find_all('startdatetime').decompose()
soup.find_all('enddatetime').decompose()
soup.find_all('lastmodified').decompose()
'''

"\nsoup.find_all('startdatetime').decompose()\nsoup.find_all('enddatetime').decompose()\nsoup.find_all('lastmodified').decompose()\n"

In [20]:
soup.startdatetime.clear()
soup.enddatetime.clear()
soup.lastmodified.clear()

### Dagsordenpunkter uden tags (lokal)

In [22]:
punkter = []  # punkterne uden tags 

for dagsordenpunkt in range(0, len(dagsordenpunkter)):
    punkter.append(dagsordenpunkter[dagsordenpunkt].get_text(separator = ' '))

In [23]:
punkter[0]

'0 FM Punkt 0 Meddelelser fra formanden Holger K. Nielsen SF formand Tredje næstformand  (Holger K. Nielsen) Typeset Mødet er åbnet. Finansudvalget har afgivet: Betænkning og indstilling vedrørende forslag til folketingsbeslutning i henhold til grundlovens § 47 med hensyn til statsregnskabet for finansåret 2008. (Beslutningsforslag nr. B 248). Betænkningen og indstillingen vil fremgå af www.folketingstidende.dk.'

### Meta data (lokal)

In [25]:
dato = bs(xml).find("dateofsitting").text[:10]
moedenr = bs(xml).find("meetingnumber").text
samling = bs(xml).find("parliamentarysession").text
hvem = bs(xml).find("parliamentarygroup").text
lokation = bs(xml).find("location").text

In [26]:
lokation

'Folketingssalen'

### List of filepaths (skrottet)

In [28]:
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles 

#getListOfFiles('all')

In [29]:
path_list = getListOfFiles('sample')

In [66]:
path_list

['sample/20171_M11_helemoedet.xml',
 'sample/20211_M60_helemoedet.xml',
 'sample/20171_M46_helemoedet.xml']

# Extract metadata function

In [69]:
#def extract_metadata(file): 
    #tree = ET.parse(file)
    #root = tree.getroot()
    #xml = ET.tostring(root, encoding='utf8').decode('utf8')

def extract_metadata(xml): 
    soup = bs(xml, 'lxml')
    
    d = soup.find("dateofsitting").text[:10]
    m = soup.find("meetingnumber").text
    s = soup.find("parliamentarysession").text
    g = soup.find("parliamentarygroup").text
    l = soup.find("location").text
    
    date.append(d)
    meetingnumber.append(m)
    session.append(s)
    group.append(g)
    location.append(l)

In [70]:
# empty lists (columns)
date = []
content = []
meetingnumber = []
session = []
group = []
location = []

# Content function 

In [88]:
''' lokalt
def extract_content(file):
    tree = ET.parse(file)
    root = tree.getroot()
    xml = ET.tostring(root, encoding='utf8').decode('utf8')
'''    

def extract_content(xml):
    soup = bs(xml, 'lxml')
    
    dp = soup.find_all("dagsordenpunkt")
    
    for punkt in range(0, len(dp)):
        content.append(dp[punkt].get_text(separator = ' '))

# Apply content function to files

In [37]:
extract_content('20091_M100_helemoedet.xml')

In [38]:
dp = soup.find_all("dagsordenpunkt")
    
for punkt in range(0, len(dp)):
    content.append(dp[punkt].get_text(separator = ' '))

In [87]:
len(content)

0

In [40]:
t = extract_content('20091_M100_helemoedet.xml')

In [41]:
for path in path_list: 
    extract_content(path)

# Apply metadata function to files

In [43]:
#for path in path_list: 
#    extract_metadata(path)

In [85]:
test = indhold[0:5]
for i in test: 
    extract_metadata(i)

In [86]:
session

['20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091',
 '20091']

# Make DataFrame

In [46]:
df = pd.DataFrame([date, content, meetingnumber, session, group, location]).transpose()
df.columns = ['date', 'content', 'meetingnumber', 'session', 'group', 'location']
df

,date,content,meetingnumber,session,group,location
0,2017-10-27,0 FM Punkt 0 Meddelelser fra formanden Holger ...,11,20171,Folketinget,Folketingssalen
1,2022-02-10,1 222 L BEH1 1. behandling af L 222: Om nedsæt...,60,20211,Folketinget,Folketingssalen
2,2018-01-18,2 218 L BEH1 1. behandling af L 218: Om nedsæt...,46,20171,Folketinget,Folketingssalen
3,None,3 219 L BEH1 1. behandling af L 219: Om forhøj...,None,None,None,None
4,None,4 223 L BEH1 1. behandling af L 223: Om staten...,None,None,None,None
5,None,5 221 L BEH1 1. behandling af L 221: Om person...,None,None,None,None
6,None,6 220 L BEH1 1. behandling af L 220: Om nedsæt...,None,None,None,None
7,None,0 FM Punkt 0 Meddelelser fra formanden Holger ...,None,None,None,None
